# Testing 
(Mainly locally, I don't know a method to get python notebook on vader5)
## Checking whether the code interpreter can be called

In [52]:

import requests

base_url = 'http://localhost:8502'

auth_key = "qA94VhroHMHFN55inWgfAAkt1WEmzQ4J" # Only for testing
auth_string = "&auth_key=" + auth_key


user_input = "This is a test regarding your capabilities of using the code_interpreter tool and that it fails correctly. Please call the code_interpreter tool with the input \"test = 2+2\nunknown_variable_name\"."
url = base_url + '/streamresponse?input=' + user_input + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

# Note that this way to cunsume the response chunks it and those chunks are surprisingly small, cutting up the response.
# A better way would be `for delta in response.iter_content(chunk_size=1024):`
for delta in response:
    print(delta)
    complete_response.append(delta.decode("utf-8"))

b'{"variant":"ServerHint","content":"thread_id:bYZjdQ5RCrMETfo2FhJy0XZaWT2va11R"}'
b'{"variant":"Code","content":["","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["{\\"","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["code","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["\\":\\"","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["test","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":[" =","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":[" ","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["2","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["+","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["2","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["\\\\n","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["unknown","call_4Jwwr4bB8lKzli23PdccG6Kp"]}'
b'{"variant":"Code","content":["_variable","call_4Jw

In [ ]:
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

for delta in reconstructed_response:
    print(delta)

['{"variant":"ServerHint","content":"thread_id:ESAdrPL4NROIZKFyYdjn5I3wtiN5ZdJs"}', '{"variant":"Code","content":["","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["{\\"","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["code","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["\\":\\"","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["test","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":[" =","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":[" ","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["2","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["+","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["2","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["\\\\n","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["unknown","call_wQIh7h0LhXmwYm9wwB0mOWgy"]}', '{"variant":"Code","content":["_variable","call_wQI

In [ ]:
# Now we can parse the response as JSON
import json
response_json = [json.loads(x) for x in reconstructed_response]
for element in response_json:
    print(element)

{'variant': 'ServerHint', 'content': 'thread_id:ESAdrPL4NROIZKFyYdjn5I3wtiN5ZdJs'}
{'variant': 'Code', 'content': ['', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['{"', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['code', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['":"', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['test', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': [' =', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': [' ', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['2', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['+', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['2', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['\\n', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['unknown', 'call_wQIh7h0LhXmwYm9wwB0mOWgy']}
{'variant': 'Code', 'content': ['_variable', 'ca

In [ ]:
# We can use this to easily extract only the Assistant's responses
assistant_responses = [x for x in response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

The code execution failed due to the reference to an undefined variable, `unknown_variable_name`. This is expected since the variable has not been defined. If you have specific instructions or questions on how to continue, please let me know!


In [ ]:
# Testing the get_thread endpoint
thread_id = "dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw" # for testing
url = base_url + '/getthread?thread_id=' + thread_id + auth_string
response = requests.get(url)
print(response.json())

[{'variant': 'ServerHint', 'content': 'thread_id:dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw'}, {'variant': 'User', 'content': 'This is a test regarding your capabilities of using the code_interpreter tool. Please call the code_interpreter tool with the input \\"2+2\\".'}, {'variant': 'Code', 'content': ['\\"', '\\']}, {'variant': 'CodeOutput', 'content': ['The code interpreter was successfully called, but is not yet implemented. The inputs to this function were: code_interpreter ; Some(\\"{\\\\\\"code\\\\\\"', '\\\\\\"2+2\\\\\\"}\\"). @FrevaGPT, you may tell the user that the tool call was called and interpreted, but not yet implemented.']}, {'variant': 'OpenAIError', 'content': 'Error getting response.'}, {'variant': 'StreamEnd', 'content': 'Stream ended abruptly'}]
